#### Prerequisites


In [1]:
from collections import defaultdict, deque
from typing import List, Optional


class TreeNode:
    def __init__(self, val: int = 0, left=None, right=None) -> None:
        self.val = val
        self.left = left
        self.right = right

    def __str__(self) -> str:
        s = ""
        lines, *_ = self._display_aux()
        for line in lines:
            s += line + "\n"
        return s

    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = "%s" % self.val
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = "%s" % self.val
            u = len(s)
            first_line = (x + 1) * " " + (n - x - 1) * "_" + s
            second_line = x * " " + "/" + (n - x - 1 + u) * " "
            shifted_lines = [line + u * " " for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = "%s" % self.val
            u = len(s)
            first_line = s + x * "_" + (n - x) * " "
            second_line = (u + x) * " " + "\\" + (n - x - 1) * " "
            shifted_lines = [u * " " + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = "%s" % self.val
        u = len(s)
        first_line = (x + 1) * " " + (n - x - 1) * "_" + s + y * "_" + (m - y) * " "
        second_line = (
            x * " " + "/" + (n - x - 1 + u + y) * " " + "\\" + (m - y - 1) * " "
        )
        if p < q:
            left += [n * " "] * (q - p)
        elif q < p:
            right += [m * " "] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * " " + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2


def create_binary_tree_from_list(values: List[Optional[int]]) -> Optional[TreeNode]:
    if not values:
        return None

    root = TreeNode(values[0])
    queue = [root]
    i = 1

    while i < len(values):
        current = queue.pop(0)

        if values[i] is not None:
            current.left = TreeNode(values[i])
            queue.append(current.left)
        i += 1

        if i < len(values) and values[i] is not None:
            current.right = TreeNode(values[i])
            queue.append(current.right)
        i += 1

    return root

## 979. Distribute Coins in Binary Tree

    Difficulty - Medium
    Topic - Binary Tree
    Algo - DFS

You are given the `root` of a binary tree with `n` nodes where each `node` in the tree has `node.val` coins. There are `n` coins in total throughout the whole tree.

In one move, we may choose two adjacent nodes and move one coin from one node to another. A move may be from parent to child, or from child to parent.

Return _the **minimum** number of moves required to make every node have **exactly** one coin_.

**Constraints:**

-   The number of nodes in the tree is `n`.
-   `1 <= n <= 100`
-   `0 <= Node.val <= n`
-   The sum of all `Node.val` is `n`.


In [17]:
class Solution:
    def distributeCoins(self, root: Optional[TreeNode]) -> int:
        total_steps = 0

        def dfs(node: Optional[TreeNode]) -> int:
            nonlocal total_steps

            if not node:
                return 0

            left_balance = dfs(node.left)
            right_balance = dfs(node.right)

            total_steps += abs(left_balance) + abs(right_balance)

            return node.val + left_balance + right_balance - 1

        dfs(root)
        return total_steps


if __name__ == "__main__":
    sol = Solution()
    cases = [
        {"root": [3, 0, 0]},
        {"root": [0, 3, 0]},
        {"root": [0, 0, 2, 4, 0, 1, 0]},
    ]
    for case in cases:
        root = create_binary_tree_from_list(case["root"])
        print(root)
        print(sol.distributeCoins(root))
        print("---------------------------------------------------------")

 3 
/ \
0 0

2
---------------------------------------------------------
 0 
/ \
3 0

3
---------------------------------------------------------
  _0_  
 /   \ 
 0   2 
/ \ / \
4 0 1 0

6
---------------------------------------------------------


## 992. Subarrays with K Different Integers

    Difficulty - Hard
    Topics - Array
    Algos - Hash Table, Sliding Window

Given an integer array `nums` and an integer `k`, return _the number of **good subarrays** of_ `nums`.

A **good array** is an array where the number of different integers in that array is exactly `k`.

-   For example, `[1,2,3,1,2]` has `3` different integers: `1`, `2`, and `3`.

A **subarray** is a **contiguous** part of an array.

**Constraints:**

-   <code>1 <= nums.length <= 2 \* 10<sup>4</sup></code>
-   `1 <= nums[i], k <= nums.length`


In [18]:
class Solution:
    def subarraysWithAtMostKDistinct(self, nums: list[int], k: int) -> int:
        memo = defaultdict(int)
        ans = left = 0

        for right, num in enumerate(nums):
            memo[num] += 1
            while len(memo) > k:
                memo[nums[left]] -= 1
                if memo[nums[left]] == 0:
                    del memo[nums[left]]
                left += 1
            ans += right - left + 1

        return ans

    def subarraysWithKDistinct(self, nums: list[int], k: int) -> int:
        return self.subarraysWithAtMostKDistinct(
            nums, k
        ) - self.subarraysWithAtMostKDistinct(nums, k - 1)


if __name__ == "__main__":
    sol = Solution()
    cases = [{"nums": [1, 2, 1, 2, 3], "k": 2}, {"nums": [1, 2, 1, 3, 4], "k": 3}]
    for case in cases:
        print(sol.subarraysWithKDistinct(case["nums"], case["k"]))

7
3


## 995. Minimum Number of K Consecutive Bit Flips

    Difficulty - Hard
    Topics - Array, Queue
    Algos - Bit Manipulation, Sliding Window, Prefix Sum

You are given a binary array `nums` and an integer `k`.

A **k-bit flip** is choosing a **subarray** of length `k` from `nums` and simultaneously changing every `0` in the subarray to `1`, and every `1` in the subarray to `0`.

Return _the minimum number of **k-bit flips** required so that there is no_ `0` _in the array_. If it is not possible, return `-1`.

A **subarray** is a **contiguous** part of an array.

**Constraints:**

-   <code>1 <= nums.length <= 10<sup>5</sup></code>
-   `1 <= k <= nums.length`


In [2]:
class Solution:
    def minKBitFlips(self, nums: List[int], k: int) -> int:
        n = len(nums)
        flip_count = 0
        queue = deque()

        for i in range(n):
            # Remove flips that are out of the window of size k
            if queue and queue[0] <= i - k:
                queue.popleft()

            # Determine if we need to flip at this position
            if len(queue) % 2 == nums[i]:
                # If there are not enough elements to flip, return -1
                if i + k > n:
                    return -1
                # Add this flip to the queue
                queue.append(i)
                flip_count += 1

        return flip_count


if __name__ == "__main__":
    sol = Solution()
    cases = [
        {"nums": [0, 1, 0], "k": 1},
        {"nums": [1, 1, 0], "k": 2},
        {"nums": [0, 0, 0, 1, 0, 1, 1, 0], "k": 3},
    ]
    for case in cases:
        print(sol.minKBitFlips(case["nums"], case["k"]))

2
-1
3
